# Yelp API Restaurant Calls By City

For each city listed in cities.csv, return all restaurants found via yelp api and
extract data regarding:

- name
- address & zip
- coordinates
- rating
- review count
- price level
- category
- yelp id


From above data, also determine the following for each city:

- total count of restaurants
- distribution of categories (eg. 10% Pizza, 20% Sushi, etc.)
- ratio of price level options (ie. total count in upper level divided by count in lower level)
- total count of restaurants above a given rating
- concentration of restaurants (ie. total count divided by sq. mi of city)


In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys import api_key

### Perform API calls

- Read cities.csv
- Build API call and process each city
- Extract and write data to new csv

In [5]:
# Create dataframe from csv file
cities_df = pd.read_csv("cities.csv", names=["City", "County", "Population", "Area (sq. mi)"])
cities_df

,City,County,Population,Area (sq. mi)
0,Alameda,Alameda,"73,812",10.61
1,Albany,Alameda,"18,539",1.79
2,American Canyon,Napa,"19,454",4.84
3,Antioch,Contra Costa,"102,372",28.35
4,Atherton,San Mateo,"6,914",5.02
5,Belmont,San Mateo,"25,835",4.62
6,Belvedere,Marin,"2,068",0.52


In [34]:
# Function returns up to 1000 restaurant listings for input city
def get_restaurants(city, api_key):
    
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % api_key}
    restaurant_data = []
    
    # Increases search return limit
    for offset in range(0, 1000, 50):
        
        # Set parameters and pass into API calls
        params = {"term": "restaurants", "location":city + ", CA", "limit":50, "offset":offset}
        req = requests.get(url, params=params, headers=headers)
        
        # Check for errors in status code
        if req.status_code == 200:
            response = req.json()
            if response["businesses"] == []:
                break
            else:
                restaurant_data.append(response["businesses"])
                total = response["total"]
        elif req.status_code == 400:
            break
    
    with open('restaurant_data.csv', 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(restaurant_data)
    csvFile.close()

    return total, restaurant_data

In [36]:
get_restaurants("Albany", api_key)

(227,
 [[{'id': 'g15dMYbefEL-ylCgk0MBbw',
    'alias': 'zaytoon-mediterranean-restaurant-and-bar-albany-2',
    'name': 'Zaytoon Mediterranean Restaurant & Bar',
    'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/lVeK1z9npSJ3dW7tWGg6ww/o.jpg',
    'is_closed': False,
    'url': 'https://www.yelp.com/biz/zaytoon-mediterranean-restaurant-and-bar-albany-2?adjust_creative=BY3Dl1B7yuLAsZtIFLov6A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=BY3Dl1B7yuLAsZtIFLov6A',
    'review_count': 318,
    'categories': [{'alias': 'mediterranean', 'title': 'Mediterranean'},
     {'alias': 'mideastern', 'title': 'Middle Eastern'},
     {'alias': 'bars', 'title': 'Bars'}],
    'rating': 4.5,
    'coordinates': {'latitude': 37.89052, 'longitude': -122.29773},
    'transactions': ['restaurant_reservation'],
    'price': '$$',
    'location': {'address1': '1133 Solano Ave',
     'address2': '',
     'address3': '',
     'city': 'Albany',
     'zip_code': '94706',
     'country'

In [37]:
restaurant_df = pd.read_csv("restaurant_data.csv")
restaurant_df

ParserError: Error tokenizing data. C error: Expected 16 fields in line 455, saw 50
